In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from datetime import datetime, timedelta, date
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.ensemble import GradientBoostingRegressor
from sklearn import linear_model
from sklearn.metrics import r2_score, mean_squared_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [17]:
import os
import functools as ft
files = os.listdir('../Dataset/weather_merged_2')
dfs=[]
for file in files:
    dftmp = pd.read_csv('../Dataset/weather_merged_2'+'/'+file)
    # dfs.append(dftmp.sample(n=5000))
    dftmp.drop('year', axis=1, inplace=True)
    dftmp.drop('month', axis=1, inplace=True)
    # dftmp.drop('REPORT_ID', axis=1, inplace=True)
    dfs.append(dftmp)
# df = pd.concat(dfs)

In [6]:
# corr = df.corr()
# f, ax = plt.subplots(figsize=(16, 7))
# sns.heatmap(corr, annot=True, fmt='.2f', vmin=-1, vmax=1, square=True, linewidths=1)
# f.suptitle('Heatmap showing the correlation of data attributes', fontsize=20)
# plt.show()

In [7]:
def make_metrics(models):
    data = {
        'name': [model.name for model in models[1:]],
        'r2': [model.r2 for model in models[1:]],
        'rmse': [model.rmse for model in models[1:]]
    }
    data['name'] = 'average R2 and sum RMSE'
    data['r2'].append(np.mean(data['r2']))
    data['rmse'].append(np.sum(data['rmse']))
    return pd.DataFrame(data)

In [8]:
class Model:
  def __init__(self, name, data, predict_features, test_size, ml_model):
    self.name = name
    self.data = data
    self.predict_features = predict_features
    self.is_trained = False
    self.test_size = test_size
    self.ml_model = ml_model
    self.do_things()

  def cal_rmse(self):
    self.rmse = mean_squared_error(self.ytest, self.ypredict, squared=False)
    return self.rmse

  def prequisite(self, test_size):
    self.features = [i for i in self.data.columns if i != self.predict_features]
    self.X = self.data[self.features].values
    self.y = self.data[self.predict_features].values
    self.Xtrain, self.Xtest, self.ytrain, self.ytest = train_test_split(self.X, self.y, test_size=test_size)
    return None

  def fit(self):
    self.is_trained = True
    self.ml_model.fit(self.Xtrain, self.ytrain)
    self.ypredict = self.ml_model.predict(self.Xtest)
    return self.ml_model

  def cal_r2_score(self):
    self.r2 = r2_score(self.ytest, self.ypredict)
    return self.r2

  def do_things(self) -> None:
    self.prequisite(self.test_size)
    self.fit()
    self.cal_rmse()
    self.cal_r2_score()
    return None

  def __repr__(self) -> str:
    if not self.is_trained:
      return f'<{self.name}> (is not trained yet)>'
    return f'<({self.name}: [R² Score: {self.r2}], [RMSE: {self.rmse}])>'

In [9]:
def getreport_id (df):
    return df['REPORT_ID'].unique()

In [10]:
models = [None]
# list_report_id = getreport_id(df)
for i in range(0,len(dfs)):
    # report_id = dfs[i]['REPORT_ID'][0]
    models += [
        Model(
            ml_model=linear_model.LinearRegression(),
            name=f'Dataset of report {i}',
            data=dfs[i],
            predict_features='vehicleCount',
            test_size=1/4
        )
    ]
    
make_metrics(models)

,name,r2,rmse
0,average R2 and sum RMSE,0.514861,36.165153
1,average R2 and sum RMSE,0.145451,49.394590
2,average R2 and sum RMSE,0.569667,24.864471
3,average R2 and sum RMSE,0.450906,25.519048
4,average R2 and sum RMSE,0.077243,160.895614
...,...,...,...
445,average R2 and sum RMSE,0.067750,3.206678
446,average R2 and sum RMSE,0.001042,2.647879
447,average R2 and sum RMSE,0.009141,2.617543
448,average R2 and sum RMSE,0.065004,2.164117


In [11]:
#make histogram
def make_histogram(df, feature):
    df['r2']= df['r2'].apply(lambda x: x*100)
    df = df[df['r2']>0]
    plt.figure(figsize=(12, 6))
    plt.hist(df[feature], bins=100)
    plt.title(f'Histogram of {feature}', fontsize=20)
    plt.xlabel(feature, fontsize=15)
    plt.ylabel('Frequency', fontsize=15)
    plt.show()

In [12]:
# make_histogram(make_metrics(models), 'r2')

In [13]:
df_predict= make_metrics(models)

In [15]:
df_predict.sort_values(by=['r2'], inplace=True, ascending=False)
df_predict.head(3)

,name,r2,rmse
238,average R2 and sum RMSE,0.688277,73.850212
237,average R2 and sum RMSE,0.680464,64.694049
267,average R2 and sum RMSE,0.675037,33.587072


In [16]:
dfs[238].head(3)

,day,hour,avgMeasuredTime,vehicleCount,avgSpeed
0,1,7,72.000000,40,60.000000
1,1,8,81.416667,235,53.583333
2,1,9,79.166667,234,56.000000
